In [11]:
# train_model.py or train_model.ipynb

from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer
)
from datasets import load_dataset
import evaluate, numpy as np

# 🧠 Force fallback to eager mode on Windows
import torch._dynamo as dynamo
dynamo.config.suppress_errors = True

# 1️⃣ Load and tokenize dataset
ds = load_dataset("glue", "sst2")
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-mini")
ds = ds.map(lambda x: tokenizer(x["sentence"], truncation=True, padding="max_length"), batched=True)
ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
train_ds, eval_ds = ds["train"], ds["validation"]

# 2️⃣ Initialize model
model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-mini", num_labels=2)

# 3️⃣ Metrics
acc = evaluate.load("accuracy")
f1 = evaluate.load("f1")
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": acc.compute(predictions=preds, references=p.label_ids)["accuracy"],
        "f1": f1.compute(predictions=preds, references=p.label_ids)["f1"],
    }

# 4️⃣ TrainingArguments — no compile flags
training_args = TrainingArguments(
    output_dir="results",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    fp16=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    warmup_steps=50,
    gradient_checkpointing=True,
    optim="adamw_torch_fused"
)

# 5️⃣ Instantiate Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 6️⃣ Train and save
trainer.train()
trainer.save_model("sentiment_model")
tokenizer.save_pretrained("sentiment_model")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-mini and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\dell\AppData\Local\Temp\ipykernel_13752\940371206.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\dell\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\dell\anaconda3\Lib\site-packages\transformers\utils\generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.270600,0.394557,0.852064,0.854893
2,0.208300,0.452929,0.854358,0.857143


c:\Users\dell\anaconda3\Lib\site-packages\transformers\utils\generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
c:\Users\dell\anaconda3\Lib\site-packages\transformers\utils\generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor'

('sentiment_model\\tokenizer_config.json',
 'sentiment_model\\special_tokens_map.json',
 'sentiment_model\\vocab.txt',
 'sentiment_model\\added_tokens.json',
 'sentiment_model\\tokenizer.json')